In [35]:
import numpy as np
import pandas as pd
import sys
import re
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
model_path = '/home/lambert/Git/Titanic/Titanic_model/'

In [36]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [37]:
import re
def get_title(name):
    if pd.isnull(name):
        return 'Null'
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return 'None'

In [38]:
titles = {'mr':1, 'mrs':2, 'mme':2, 'ms':3, 'miss':3, 'mlle':3, 'don':4, 'sir':4, 'jonkheer':4,
          'major':4, 'col':4, 'dr':4, 'master':4, 'capt':4, 'dona':5, 'lady':5, 'countess':5,
         'rev':6}
df_train['Title'] = df_train['Name'].apply(lambda name: titles.get(get_title(name)))
df_train['Honor'] = df_train['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)
df_test['Title'] = df_test['Name'].apply(lambda name: titles.get(get_title(name)))
df_test['Honor'] = df_test['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)

In [39]:
# 1. Data clearning
# df_train.Age.fillna(fill_df_Age, inplace=True)
# df_test.Age.fillna(fill_df_Age, inplace=True)
for i in range(1,7):
    a = df_train[(df_train.Age.isnull()) & (df_train.Title==i)]
    df_train.Age.iloc[a.index] = df_train.Age[df_train.Title==i].median()
    b = df_test[(df_test.Age.isnull()) & (df_test.Title==i)]
    df_test.Age.iloc[b.index] = df_train.Age[df_train.Title==i].median()

# Cabin - fill nan with 'fill' and select first cabin
df_train.Cabin.fillna('fill',inplace=True)
df_train.Cabin = df_train.Cabin.map(lambda x:x.split(' ')[0])
df_test.Cabin.fillna('fill', inplace=True)
df_test.Cabin = df_test.Cabin.map(lambda x:x.split(' ')[0])

# Sex - male: 1, female: 1
df_train.Sex.replace({'male':1, 'female':0}, inplace=True)
df_test.Sex.replace({'male':1, 'female':0}, inplace=True)

df_train = df_train.dropna(axis=0, how='any')
df_test.Fare.fillna(df_test.Fare.mean(), inplace=True)

# Create a 'Deceased' column for sconed class
df_train['Deceased'] = df_train.Survived.apply(lambda s: int(not s))

/home/lambert/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [40]:
# 2. Select the features
sFeatures = ['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',             
 'Cabin',             
 'Embarked',
 'Title',
 'Honor']

X = df_train[sFeatures]
y = df_train[['Survived','Deceased']]
y = y.values.reshape(len(y),2)
X_test = df_test[sFeatures]

In [41]:
# 3. Label Encoder
# X_test has elements that X doesn't have
# create a Cabin_labels cover X and X_test
X_Cabin_unique = X['Cabin'].unique()
X_lack = X_test.Cabin[X_test['Cabin'].isin(X_Cabin_unique)==0].values
Cabin_labels = np.append(X_Cabin_unique, X_lack)
# print(X_Cabin_unique.shape, X_Cabin_unique)
# print(Cabin_labels.shape, Cabin_labels)

le_Cabin = LabelEncoder().fit(Cabin_labels)
le_Embarked = LabelEncoder().fit(np.array(X['Embarked'].tolist()))

X.loc[:,'Cabin'] = le_Cabin.transform(X['Cabin'])
X.loc[:,'Embarked'] = le_Embarked.transform(X['Embarked'])
X_test.loc[:,'Cabin'] = le_Cabin.transform(X_test['Cabin'])
X_test.loc[:,'Embarked'] = le_Embarked.transform(X_test['Embarked'])
X = X.astype(np.float32)
y = y.astype(np.float32)

/home/lambert/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [42]:
# Validation
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [43]:
# No Validation
X_train = X
y_train = y

In [59]:
X_train.shape

(889, 10)

### Tensor Graph

In [46]:
def next_batch(batch, X, y):
    num = np.arange(len(X))
    np.random.shuffle(num)
    select = num[0:batch]
    x_batch = X.iloc[select]
    y_batch = y[select]
    
    return(x_batch, y_batch)

In [47]:
batch_size = 200

def optimize(num_iterations):
    total_loss = 0
    for i in range(num_iterations):
        x_batch, y_batch = next_batch(batch_size, X_train_data, y_train_data)
        feed_dict_train = {X_place: x_batch, y_place: y_batch}
        _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
        total_loss += loss
        print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
    print ('Training complete!')

In [48]:
# Tensorflow variables
# with tf.device('/device:GPU:0'):
features= len(sFeatures)
X_place = tf.placeholder(tf.float32, shape=[None,features])
y_place = tf.placeholder(tf.float32, shape=[None,2])
W = tf.Variable(tf.random_normal([features, 2]), name='weights')
b = tf.Variable(tf.zeros(2), name='bias')
logits = tf.matmul(X_place, W) + b
y_pred = tf.nn.softmax(logits)

In [49]:
# Cross Entropy
# with tf.device('/device:GPU:0'):
learning_rate = 0.001
cross_entropy = -tf.reduce_sum(y_place * tf.log(y_pred + 1e-10), axis=1)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_train)
cost = tf.reduce_mean(cross_entropy)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
saver = tf.train.Saver()

In [50]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [51]:
# # training starting option 1
# for epoch in range(10000):
#     total_loss = 0.
#     x_batch, y_batch = next_batch(batch_size, X_train, y_train)
#     feed_dict_train = {X_place: x_batch, y_place: y_batch}
#     _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
#     total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [52]:
# sess.close()

In [53]:
# training starting option 1 with CPU
# for epoch in range(100):
#     total_loss = 0.
#     for i in range(len(X_train)):
#         feed = {X_place: X_train, y_place: y_train}
#         _, loss = sess.run([train_op, cost], feed_dict=feed)
#         total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [54]:
# training starting option 1 with GPU
# with tf.device('/device:GPU:0'):
start = time.time()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device('/device:GPU:0'):
        sess.run(tf.global_variables_initializer())
        for epoch in range(10000):
            total_loss = 0.
            for i in range(len(X_train)):
                feed = {X_place: X_train, y_place: y_train}
                _, loss = sess.run([train_op, cost], feed_dict=feed)
                total_loss += loss
            print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
        #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
        #     print('predict: {:.6f}'.format(y_pred))
        print ('Training complete!')

        feed_dict_train = {X_place: X_train}
        prediction = sess.run(y_pred, feed_dict=feed_dict_train)
        correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
        # evaluate the accuracy
        accuracy = np.mean(correct)
        print('Accuracy on validation: {:2.2%}'.format (accuracy))
    save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
end = time.time()
print('time spend: {:.4f} sec'.format(end-start))

Epoch: 0001, total loss=7741.516098022
Epoch: 0002, total loss=7738.829321861
Epoch: 0003, total loss=7734.809428215
Epoch: 0004, total loss=7731.230428696
Epoch: 0005, total loss=7730.431954384
Epoch: 0006, total loss=7730.153929710
Epoch: 0007, total loss=7729.909674644
Epoch: 0008, total loss=7729.665932655
Epoch: 0009, total loss=7729.419364929
Epoch: 0010, total loss=7729.170191765
Epoch: 0011, total loss=7728.918432236
Epoch: 0012, total loss=7728.664295197
Epoch: 0013, total loss=7728.407405853
Epoch: 0014, total loss=7728.149026871
Epoch: 0015, total loss=7727.888969421
Epoch: 0016, total loss=7727.626473427
Epoch: 0017, total loss=7727.363004684
Epoch: 0018, total loss=7727.097887993
Epoch: 0019, total loss=7726.830873489
Epoch: 0020, total loss=7726.563403130
Epoch: 0021, total loss=7726.293632507
Epoch: 0022, total loss=7726.022218704
Epoch: 0023, total loss=7725.749572754
Epoch: 0024, total loss=7725.473988533
Epoch: 0025, total loss=7725.196638107
Epoch: 0026, total loss=7

Epoch: 0212, total loss=1781.671990097
Epoch: 0213, total loss=1546.757864773
Epoch: 0214, total loss=1535.627161920
Epoch: 0215, total loss=1599.735845983
Epoch: 0216, total loss=1726.638076544
Epoch: 0217, total loss=1646.015600920
Epoch: 0218, total loss=1562.008459568
Epoch: 0219, total loss=1746.996820986
Epoch: 0220, total loss=1669.314358413
Epoch: 0221, total loss=1553.161132872
Epoch: 0222, total loss=1579.990258694
Epoch: 0223, total loss=1675.051711857
Epoch: 0224, total loss=1578.185362399
Epoch: 0225, total loss=1670.126489937
Epoch: 0226, total loss=1642.595809162
Epoch: 0227, total loss=1611.094059944
Epoch: 0228, total loss=1542.142427802
Epoch: 0229, total loss=1724.765162468
Epoch: 0230, total loss=1624.044099450
Epoch: 0231, total loss=1648.115971446
Epoch: 0232, total loss=1626.583009720
Epoch: 0233, total loss=1603.301566482
Epoch: 0234, total loss=1611.670577884
Epoch: 0235, total loss=1702.244235218
Epoch: 0236, total loss=1558.563106477
Epoch: 0237, total loss=1

Epoch: 0423, total loss=1643.140432596
Epoch: 0424, total loss=1617.095986426
Epoch: 0425, total loss=1529.978061855
Epoch: 0426, total loss=1641.973672092
Epoch: 0427, total loss=1636.433665276
Epoch: 0428, total loss=1602.335602462
Epoch: 0429, total loss=1524.038945079
Epoch: 0430, total loss=1593.315794408
Epoch: 0431, total loss=1643.267981529
Epoch: 0432, total loss=1559.135881722
Epoch: 0433, total loss=1607.178144157
Epoch: 0434, total loss=1439.579446077
Epoch: 0435, total loss=1637.833487093
Epoch: 0436, total loss=1684.242131710
Epoch: 0437, total loss=1479.570369840
Epoch: 0438, total loss=1737.845285356
Epoch: 0439, total loss=1485.663780630
Epoch: 0440, total loss=1522.387553930
Epoch: 0441, total loss=1584.848053694
Epoch: 0442, total loss=1609.541321218
Epoch: 0443, total loss=1579.547386408
Epoch: 0444, total loss=1625.552997887
Epoch: 0445, total loss=1643.068936944
Epoch: 0446, total loss=1527.872392058
Epoch: 0447, total loss=1637.344266653
Epoch: 0448, total loss=1

Epoch: 0634, total loss=1583.579483151
Epoch: 0635, total loss=1415.534041345
Epoch: 0636, total loss=1564.271731734
Epoch: 0637, total loss=1646.785235822
Epoch: 0638, total loss=1537.895409346
Epoch: 0639, total loss=1498.678470075
Epoch: 0640, total loss=1558.445600390
Epoch: 0641, total loss=1620.179279625
Epoch: 0642, total loss=1562.902453482
Epoch: 0643, total loss=1446.033628047
Epoch: 0644, total loss=1608.345897853
Epoch: 0645, total loss=1492.993066967
Epoch: 0646, total loss=1416.655626595
Epoch: 0647, total loss=1598.967617393
Epoch: 0648, total loss=1504.276171982
Epoch: 0649, total loss=1464.020626724
Epoch: 0650, total loss=1580.372054875
Epoch: 0651, total loss=1615.229103208
Epoch: 0652, total loss=1394.223473668
Epoch: 0653, total loss=1548.419037938
Epoch: 0654, total loss=1572.354772449
Epoch: 0655, total loss=1382.788058877
Epoch: 0656, total loss=1585.430456936
Epoch: 0657, total loss=1565.240519762
Epoch: 0658, total loss=1570.691242337
Epoch: 0659, total loss=1

Epoch: 0845, total loss=1457.765558839
Epoch: 0846, total loss=1578.458762050
Epoch: 0847, total loss=1518.662419319
Epoch: 0848, total loss=1430.116181970
Epoch: 0849, total loss=1554.880965829
Epoch: 0850, total loss=1447.252210915
Epoch: 0851, total loss=1462.170810938
Epoch: 0852, total loss=1448.270988941
Epoch: 0853, total loss=1446.909454286
Epoch: 0854, total loss=1630.720983505
Epoch: 0855, total loss=1577.761880755
Epoch: 0856, total loss=1463.651230633
Epoch: 0857, total loss=1528.791512072
Epoch: 0858, total loss=1449.527380824
Epoch: 0859, total loss=1421.722926319
Epoch: 0860, total loss=1438.676900148
Epoch: 0861, total loss=1605.907999516
Epoch: 0862, total loss=1439.473148942
Epoch: 0863, total loss=1410.304621220
Epoch: 0864, total loss=1448.202291489
Epoch: 0865, total loss=1451.902476132
Epoch: 0866, total loss=1600.617235959
Epoch: 0867, total loss=1500.118284464
Epoch: 0868, total loss=1555.258540154
Epoch: 0869, total loss=1526.105223596
Epoch: 0870, total loss=1

...

Epoch: 9074, total loss=1428.398287833
Epoch: 9075, total loss=1335.752494395
Epoch: 9076, total loss=1234.803627908
Epoch: 9077, total loss=1320.215762496
Epoch: 9078, total loss=1279.838421404
Epoch: 9079, total loss=1314.978724957
Epoch: 9080, total loss=1333.033145249
Epoch: 9081, total loss=1276.731830895
Epoch: 9082, total loss=1302.032226920
Epoch: 9083, total loss=1293.099061549
Epoch: 9084, total loss=1415.083066463
Epoch: 9085, total loss=1245.734515846
Epoch: 9086, total loss=1316.127621591
Epoch: 9087, total loss=1302.120353520
Epoch: 9088, total loss=1354.793952882
Epoch: 9089, total loss=1200.870198131
Epoch: 9090, total loss=1306.879063606
Epoch: 9091, total loss=1371.873697042
Epoch: 9092, total loss=1270.809178889
Epoch: 9093, total loss=1297.078447580
Epoch: 9094, total loss=1290.259903967
Epoch: 9095, total loss=1197.356746972
Epoch: 9096, total loss=1437.317283690
Epoch: 9097, total loss=1280.515307963
Epoch: 9098, total loss=1299.760703146
Epoch: 9099, total loss=1

Epoch: 9285, total loss=1327.939654648
Epoch: 9286, total loss=1378.866817832
Epoch: 9287, total loss=1347.532877922
Epoch: 9288, total loss=1207.765214026
Epoch: 9289, total loss=1250.335816085
Epoch: 9290, total loss=1338.315634549
Epoch: 9291, total loss=1321.945695817
Epoch: 9292, total loss=1319.814202726
Epoch: 9293, total loss=1274.006182075
Epoch: 9294, total loss=1201.821731687
Epoch: 9295, total loss=1356.573885798
Epoch: 9296, total loss=1308.780739844
Epoch: 9297, total loss=1421.795056999
Epoch: 9298, total loss=1302.451392829
Epoch: 9299, total loss=1141.726357222
Epoch: 9300, total loss=1388.990366220
Epoch: 9301, total loss=1350.914026856
Epoch: 9302, total loss=1292.940227985
Epoch: 9303, total loss=1368.032483101
Epoch: 9304, total loss=1414.031191826
Epoch: 9305, total loss=1252.015938520
Epoch: 9306, total loss=1184.269337952
Epoch: 9307, total loss=1363.753836095
Epoch: 9308, total loss=1274.935859084
Epoch: 9309, total loss=1338.231334984
Epoch: 9310, total loss=1

Epoch: 9496, total loss=1172.942750275
Epoch: 9497, total loss=1318.463676631
Epoch: 9498, total loss=1294.099249780
Epoch: 9499, total loss=1363.562709391
Epoch: 9500, total loss=1274.019429028
Epoch: 9501, total loss=1387.399185658
Epoch: 9502, total loss=1271.288829625
Epoch: 9503, total loss=1270.507078111
Epoch: 9504, total loss=1302.796244919
Epoch: 9505, total loss=1373.909598768
Epoch: 9506, total loss=1233.319534361
Epoch: 9507, total loss=1314.719427884
Epoch: 9508, total loss=1058.208635926
Epoch: 9509, total loss=1431.923033237
Epoch: 9510, total loss=1346.887332559
Epoch: 9511, total loss=1330.153676391
Epoch: 9512, total loss=1473.894865811
Epoch: 9513, total loss=1300.025871098
Epoch: 9514, total loss=1268.105537713
Epoch: 9515, total loss=1307.566819131
Epoch: 9516, total loss=1303.629677951
Epoch: 9517, total loss=1351.998603046
Epoch: 9518, total loss=1189.205410898
Epoch: 9519, total loss=1356.085131228
Epoch: 9520, total loss=1282.400163591
Epoch: 9521, total loss=1

Epoch: 9707, total loss=1298.398052335
Epoch: 9708, total loss=1167.844446063
Epoch: 9709, total loss=1305.292741299
Epoch: 9710, total loss=1366.528239667
Epoch: 9711, total loss=1292.520776510
Epoch: 9712, total loss=1356.320661008
Epoch: 9713, total loss=1173.450212240
Epoch: 9714, total loss=1383.393930018
Epoch: 9715, total loss=1379.781330764
Epoch: 9716, total loss=1191.514668524
Epoch: 9717, total loss=1299.194482625
Epoch: 9718, total loss=1204.877129316
Epoch: 9719, total loss=1402.650610864
Epoch: 9720, total loss=1296.778613627
Epoch: 9721, total loss=1330.336899221
Epoch: 9722, total loss=1328.550134897
Epoch: 9723, total loss=1395.404653192
Epoch: 9724, total loss=1244.793619514
Epoch: 9725, total loss=1343.277208209
Epoch: 9726, total loss=1304.905560553
Epoch: 9727, total loss=1237.146634161
Epoch: 9728, total loss=1321.616838694
Epoch: 9729, total loss=1298.811232746
Epoch: 9730, total loss=1359.825750470
Epoch: 9731, total loss=1209.416361928
Epoch: 9732, total loss=1

Epoch: 9918, total loss=1323.579516470
Epoch: 9919, total loss=1260.841908753
Epoch: 9920, total loss=1309.362678051
Epoch: 9921, total loss=1392.710701942
Epoch: 9922, total loss=1233.178007007
Epoch: 9923, total loss=1248.506791770
Epoch: 9924, total loss=1341.483131945
Epoch: 9925, total loss=1286.714893103
Epoch: 9926, total loss=1329.557404816
Epoch: 9927, total loss=1284.215212166
Epoch: 9928, total loss=1223.198460162
Epoch: 9929, total loss=1304.469812810
Epoch: 9930, total loss=1343.543997824
Epoch: 9931, total loss=1318.698937297
Epoch: 9932, total loss=1298.181950986
Epoch: 9933, total loss=1432.513923585
Epoch: 9934, total loss=1371.386307895
Epoch: 9935, total loss=1169.123508990
Epoch: 9936, total loss=1283.997098029
Epoch: 9937, total loss=1339.422308743
Epoch: 9938, total loss=1299.185411811
Epoch: 9939, total loss=1273.606904209
Epoch: 9940, total loss=1353.491284192
Epoch: 9941, total loss=1227.074674129
Epoch: 9942, total loss=1218.529331923
Epoch: 9943, total loss=1

In [ ]:
# save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
# sess.close()

In [55]:
# evaluate the training data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_train = {X_place: X_train}
    prediction = sess.run(y_pred, feed_dict=feed_dict_train)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
# evaluate the accuracy
accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 82.56%


In [22]:
# evaluate the validate data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_valid = {X_place: X_val}
    prediction = sess.run(y_pred, feed_dict=feed_dict_valid)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_val,axis=1))
# evaluate the accuracy

accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 74.89%


In [56]:
# prediction for test data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_test = {X_place: X_test}
    prediction = sess.run(y_pred, feed_dict=feed_dict_test)
    y_predict = np.argmax(prediction, axis=1)
    y_predict_1 = np.argmin(prediction, axis=1)
    
# evaluate the accuracy

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt


In [58]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict_1}).set_index('PassengerId')
submission.to_csv('submission_tf_GDO_v3.csv')

In [29]:
y_predict

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,

In [31]:
y_predict_1

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [27]:
# print_accuracy()
# plot_example_errors()
# optimize(num_iterations=10)
# print_accuracy()
# plot_example_errors()
# plot_weights()

In [57]:
y_predict_1

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [67]:
model_path.split('Titanic_model')[0]

'/home/lambert/Git/Titanic/'

In [68]:
File_Writer=tf.summary.FileWriter(model_path.split('Titanic_model')[0], sess.graph)

In [ ]:
# with tf.device('/device:GPU:0'):
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:

In [ ]:
# allocate as much GPU memory based on runtime allocations
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config)

In [ ]:
# allocate 40% memory of GPU
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# session = tf.Session(config=config, ...)

In [ ]:
# Automatically choose existing in case specified one doesn't exit
# allow_soft_placement=True
# with tf.device('/device:GPU:2'):
#   a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
#   b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
#   c = tf.matmul(a, b)
# # Creates a session with allow_soft_placement and log_device_placement set
# # to True.
# sess = tf.Session(config=tf.ConfigProto(
#       allow_soft_placement=True, log_device_placement=True))

In [ ]:
sys.argv[2]

In [60]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3012004715058479259, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 18415616
 locality {
   bus_id: 1
 }
 incarnation: 1414880434492412731
 physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"]

Logistic Regression

In [ ]:
# Logistic Regression
# lr = LogisticRegression(C=1, max_iter=100, solver='lbfgs').fit(X_train, y_train)
# yp = lr.predict(X_train)
# y_predict = lr.predict(X_test)

# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission_knn.csv')
# passengerid = np.array(df_train.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': yp}).set_index('PassengerId')
# submission.to_csv('y_train_predict.csv')

Gradient Boosting Classifier

In [ ]:
# Gradient Boosting Classifier
# gbdt = GradientBoostingClassifier(n_estimators=300, max_depth=5).fit(X_train, y_train)
# y_predict = gbdt.predict(X_test)

# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission.csv')

Random forests

In [ ]:
# RandomForest
# rfc = RandomForestClassifier(n_estimators=200, max_depth=3).fit(X_train, y_train)
# y_predict = rfc.predict(X_test)
# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission_random.csv')

Stochastic Gradient Descent (SDG)

In [ ]:
# Stochastic Gradient Descent (SGD) easily scale with more than 10^5 samples and features
# sgd = SGDClassifier(alpha=0.01, max_iter=1000).fit(X_train, y_train)
# y_predict = sgd.predict(X_test)
# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission_sdg.csv')